### Trabajo Práctico Final - ERV 
## Detección de fallas en neumáticos
**Alumno:** Manuel García Tornadú

La idea del trabajo es replicar los procedimientos para la detección de fallas en neumáticos usados en el paper **"Tire appearance defect detection method via combining HOG and LBP features"** escrito por Hongbin Liu, Xinghao Jia, Chenhui Su, Hongjuan Yang and Chengdong Li


In [20]:
import os
import os.path
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Procesamiento de imágenes

In [57]:
#carpetas de ubicación del dataset
carpeta_entrenamiento_normal = Path('./training_data/normal')
carpeta_entrenamiento_cracked = Path('./training_data/cracked')
carpeta_test_normal = Path('./testing_data/normal')
carpeta_test_cracked = Path('./testing_data/cracked')

#### Funciones para el procesamiento de imágenes:

In [67]:
#Cambia la imagen a escala de grises con los parámteros que se usan en el paper
def pasa_a_escala_de_gris(imagen):
    img = np.array(imagen)
    return np.int8(img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.144)


# Modifica el tamaño de la imagen usando el método de interpolación bilinear usado en el paper
def cambia_dimension(imagen, dimension):
    imagen_escalada = imagen.resize((dimension, dimension), Image.BILINEAR)
    return imagen_escalada


# Procesa toda una carpeta de imagenes: las pasa a escala de gris y luego reduce el tamaño al pasado por parámetro.
# Guarda las imagenes en una carpeta y devuelve la ruta de la misma
def procesa_imagenes(carpeta, dim):
    lista_nombres = list(carpeta.glob(r"*.jpg"))
    nombre_carpeta = "procesados_" + str(dim)
    carpeta_procesados = carpeta / nombre_carpeta
    carpeta_procesados.mkdir(exist_ok=True)
    i = 0
    for ruta_imagen in lista_nombres:
        img = Image.open(ruta_imagen)
        if img.mode != 'L':
            imagen_array = pasa_a_escala_de_gris(img)
            imagen_escalada = cambia_dimension(Image.fromarray(imagen_array, 'L'), dim)
        else:
            imagen_escalada = cambia_dimension(img, dim)
        nombre_archivo = "imagen_" + str(i) + ".png"
        imagen_escalada.save(carpeta_procesados / nombre_archivo)
        i += 1
    return carpeta_procesados



In [66]:
carpeta_entrenamiento_normal_procesados = procesa_imagenes(carpeta_entrenamiento_normal, 64)
carpeta_entrenamiento_cracked_procesados = procesa_imagenes(carpeta_entrenamiento_cracked, 64)
carpeta_test_normal_procesados = procesa_imagenes(carpeta_test_normal, 64)
carpeta_test_cracked_procesados = procesa_imagenes(carpeta_test_cracked, 64)

In [70]:
# Carpetas con imagenes ya procesadas:
entrenamiento_normal_procesados = Path('./training_data/normal/procesados_64')
entrenamiento_cracked_procesados = Path('./training_data/cracked/procesados_64')
test_normal_procesados = Path('./testing_data/normal/procesados_64')
test_cracked_procesados = Path('./testing_data/cracked/procesados_64')